In [1]:
import sys
sys.path.append('./src')

import pandas as pd
from chatgpt_util_imports import generate_dataset

In [12]:
# Load +31k patches as compiled by `bwhitman` for `learnfm`.
fpath = 'data/compact.bin'

dataset = generate_dataset(fpath)

print(f"Num patches: {len(dataset)}")
print(dataset[0].keys())
print(dataset[0]['operators'][0].keys())

Num patches: 31380
dict_keys(['name', 'algorithm', 'feedback', 'operators'])
dict_keys(['eg_rate_level', 'left_curve', 'right_curve', 'detune', 'rate_scaling', 'kvs', 'ams', 'output_level', 'frequency_mode', 'frequency_coarse', 'frequency_fine'])


In [17]:
df = []

for patch in dataset:
    datum = {}

    datum['name'] = patch['name']
    datum['algorith'] = patch['algorithm']
    datum['feedback'] = patch['feedback']

    for n, operator in enumerate(patch['operators']):
        for key in operator.keys():
            fieldname = f"op{n}_{key}"
            datum[fieldname] = operator[key]

    df.append(datum)

df = pd.DataFrame(df)


In [24]:
dataset[0]

{'name': 'creepjaz',
 'algorithm': 5,
 'feedback': 0,
 'operators': [{'eg_rate_level': [90, 35, 20, 54, 99, 80, 0, 0, 38, 0, 99],
   'left_curve': 0,
   'right_curve': 3,
   'detune': 3,
   'rate_scaling': 7,
   'kvs': 0,
   'ams': 3,
   'output_level': 94,
   'frequency_mode': 0,
   'frequency_coarse': 1,
   'frequency_fine': 43},
  {'eg_rate_level': [90, 35, 20, 54, 99, 80, 0, 0, 0, 0, 99],
   'left_curve': 2,
   'right_curve': 3,
   'detune': 2,
   'rate_scaling': 7,
   'kvs': 0,
   'ams': 2,
   'output_level': 86,
   'frequency_mode': 0,
   'frequency_coarse': 1,
   'frequency_fine': 43},
  {'eg_rate_level': [90, 73, 0, 54, 99, 80, 0, 0, 30, 0, 98],
   'left_curve': 0,
   'right_curve': 2,
   'detune': 1,
   'rate_scaling': 7,
   'kvs': 3,
   'ams': 4,
   'output_level': 76,
   'frequency_mode': 0,
   'frequency_coarse': 15,
   'frequency_fine': 0},
  {'eg_rate_level': [90, 30, 0, 54, 99, 80, 0, 0, 0, 0, 0],
   'left_curve': 2,
   'right_curve': 0,
   'detune': 1,
   'rate_scaling'